In [9]:
from src.llm_reviewer.llm_api import make_llm_request, LLMAPIFactory
import os
import sys
from project_path import PROJECT_PATH
sys.path.insert(0, PROJECT_PATH)


service_account_path = PROJECT_PATH + "/creds/google__sa.json"
tracking_sheet_id = "1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4"
delivery_sheet_id = "1eUif5I8xhHU8fY0X9v8r2JI9hWPh7Dq_9VXpSIHwww4"


In [4]:
    llm_client = LLMAPIFactory().get()
    make_llm_request(
        llm_client,
        [{'role': 'system', 'content': "How are you today this find evening?"}],
        'gpt-4-1106-preview',
        temperature= 0.0,
        max_tokens = 4000,
        response_format = None,
        retries = 3,
        seed=42
    )

"As an AI language model, I don't have feelings or experiences, but I'm here and ready to assist you with any questions or tasks you have. How can I help you today?"

In [1]:
from src.llm_reviewer.notebook_parser import predict_role


/home/saya/chario/upstream_character_tasks/.venv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
predict_role([{'role': 'User', 'content': 'how is my favorite GPT doing today?'}, {'role':  '', 'content': 'I am fine thank you'}, {'role':'User', 'content': 'boring as always'}])

Filling out missing header...


('Assistant', None)

In [11]:

from importlib import reload
from src.llm_reviewer import notebook_reviewer
reload(notebook_reviewer)

<module 'src.llm_reviewer.notebook_reviewer' from '/home/saya/chario/upstream_character_tasks/src/llm_reviewer/notebook_reviewer.py'>

In [26]:
from src.llm_reviewer.notebook_reviewer import review_notebook
import nbformat

fn = 'src/llm_reviewer/tests/samples/10T__ml_research.ipynb'
notebook = nbformat.read(fn, as_version=4)

review = review_notebook(
    {'nb_parsed_notebook': notebook, 'file_id': fn},
    max_threads=10,
    progress_counter= None,
    verbose = 2,
)

Reviews done: 0, Reviews left after this one: 19
Reviews done: 0, Reviews left after this one: 18
Reviews done: 0, Reviews left after this one: 17
Reviews done: 0, Reviews left after this one: 16
Reviews done: 0, Reviews left after this one: 15
Reviews done: 0, Reviews left after this one: 14
Reviews done: 0, Reviews left after this one: 13
Reviews done: 0, Reviews left after this one: 12
Reviews done: 0, Reviews left after this one: 11
Reviews done: 0, Reviews left after this one: 10
Review for turn_id=3 by reviewer='code_reviewer' is done. 1 / 20 reviews completed.
Reviews done: 1, Reviews left after this one: 9
Review for turn_id=1 by reviewer='code_reviewer' is done. 2 / 20 reviews completed.
Reviews done: 2, Reviews left after this one: 8
Review for turn_id=1 by reviewer='english_reviewer' is done. 3 / 20 reviews completed.
Reviews done: 3, Reviews left after this one: 7
Review for turn_id=3 by reviewer='english_reviewer' is done. 4 / 20 reviews completed.
Reviews done: 4, Reviews

In [27]:
review

{'turns': [{'turn': [{'role': 'human',
     'steps': [{'type': 'markdown',
       'content': "Do you know the iris dataset for ml? I wanted to explore scikit learn algorithms on it. Give me the code to solve it using some algo from scikit learn. I'm in an empty folder right now. linux"}]},
    {'role': 'llm',
     'steps': [{'type': 'markdown',
       'content': 'Yes, I know the iris dataset that is commonly used as a toy dataset to showcase the work of machine learning classification algorithms.\n\nscikit-learn has multiple algorithms that are capable of "solving" it, let\'s start with a simple logistic regression.\n\nBut first, to start we will need to set up an environment with scikit-learn python package installed.\n\nAssuming that you are in a project root directory, let\'s set up a virtual environment for this task.\n\nRun the following in the command line:\n\n```\npython -m venv venv\n```\n\nThis will create a folder named venv with the virtual environment for you.\n\nNext, you 

In [16]:
review

{'turns': [{'turn': [{'role': 'llm',
     'steps': [{'type': 'markdown',
       'content': "Do you know the iris dataset for ml? I wanted to explore scikit learn algorithms on it. Give me the code to solve it using some algo from scikit learn. I'm in an empty folder right now. linux"}]},
    {'role': 'human',
     'steps': [{'type': 'markdown',
       'content': 'Yes, I know the iris dataset that is commonly used as a toy dataset to showcase the work of machine learning classification algorithms.\n\nscikit-learn has multiple algorithms that are capable of "solving" it, let\'s start with a simple logistic regression.\n\nBut first, to start we will need to set up an environment with scikit-learn python package installed.\n\nAssuming that you are in a project root directory, let\'s set up a virtual environment for this task.\n\nRun the following in the command line:\n\n```\npython -m venv venv\n```\n\nThis will create a folder named venv with the virtual environment for you.\n\nNext, you 

In [28]:
from src.llm_reviewer.turn_reviewer import load_rubric, load_system_prompt


CODE_PROMPT = load_system_prompt('aggregated_reviews_code_judge')
CODE_RUBRICS = load_rubric('code_reviewer')
LANG_PROMPT = load_system_prompt('aggregated_reviews_english_judge')
LANG_RUBRICS = load_rubric('english_reviewer')


from src.llm_reviewer.llm_api import make_llm_request, LLMAPIFactory

def get_judgment(reviewer, feedback):
    llm_client = LLMAPIFactory().get()
    if reviewer == 'code':
        prompt = CODE_PROMPT
        rubrics = CODE_RUBRICS
    elif reviewer == 'lang':
        prompt = LANG_PROMPT
        rubrics = LANG_RUBRICS
    else:
        raise ValueError("Reviewer type not recognized.")
    
    formatted_prompt = prompt.format(FEEDBACK=feedback, GRADING_RUBRICS=rubrics)
    judgment = make_llm_request(
        llm_client,
        [{'role': 'system', 'content': formatted_prompt}],
        'gpt-4-1106-preview',
        temperature= 0.0,
        max_tokens = 4000,
        response_format = {'type': "json_object"},
        retries = 3,
    )
    return judgment

# Example of running the function
#judgment = get_judgment(CODE_PROMPT, gpt_reviews_df.iloc[0]['code_feedback'])
#judgment


In [29]:
from src.llm_reviewer.notebook_reviewer import notebook_reviews_to_df, IssueLevel

df = notebook_reviews_to_df(filter(None, [review]), IssueLevel.MEDIUM)

In [30]:
df

,nb_path,code_score,lang_score,comb_feedback,code_feedback,lang_feedback
0,src/llm_reviewer/tests/samples/10T__ml_researc...,3.8,4.0,#Turn 1:\n\n## Language(4/5):\n**Medium_Issues...,#Turn 1:\n\n## Code(4/5):\n**Medium_Issues**\n...,#Turn 1:\n\n## Language(4/5):\n**Medium_Issues...


In [31]:
get_judgment('code', df.iloc[0]['code_feedback'])

{'judgment': 'The code provided across turns generally functions correctly but often lacks error handling, efficiency improvements, and necessary function definitions, indicating room for significant enhancement.',
 'score': '3'}